
... ***CURRENTLY UNDER DEVELOPMENT*** ...


## RBFs reconstruction of historical and synthetic data

inputs required: 
  * Synthetic offshore waves - emulator output
  * Sea and swell **SWAN simulated cases**

in this notebook:
  * RBF reconstruction simulated storms
  * Generation of hourly nearshore waves with Intradaily Hydrographs



### Workflow:

<div>
<img src="../resources/nb02_03.png" width="400px">
</div>



In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..', '..', '..'))

# teslakit
from teslakit.database import Database, hyswan_db
from teslakit.rbf import RBF_Reconstruction, RBF_Validation
from teslakit.waves import Aggregate_WavesFamilies, AWL, Intradaily_Hydrograph, Snell_Propagation
from teslakit.climate_emulator import Climate_Emulator
from teslakit.util.time_operations import repair_times_hourly, add_max_storms_mask



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database
p_data = r'/media/administrador/HD/Dropbox/Guam/teslakit/data'
# p_data=r'/Users/laurac/Dropbox/Guam/teslakit/data'
# offshore
db = Database(p_data)
db.SetSite('GUAM')

# climate change - S4
db_S4 = Database(p_data)
db_S4.SetSite('GUAM_CC_S4')

# hyswan simulation database
db_sim = hyswan_db(db.paths.site.HYSWAN.sim)
db_sim_S4 = hyswan_db(db_S4.paths.site.HYSWAN.sim)


## Define profile to run

In [3]:
prf=11

In [4]:
# --------------------------------------
# aux.

def fix_target_nans(subset, target):
    'remove NaN data indexes from subset and target. RBF does not handle NaNs'
    
    r_nan = target.isnull().any(axis=1)  # find any row with nans

    if r_nan.any():
        # log
        print('remove nan data found at target:')
        print(target[r_nan])
        
        # clean data
        target = target[~r_nan]
        subset = subset[~r_nan]
        
    return subset, target


# Snell propagation 400m -> 21m
def snell_prop_data(waves_dset, d1, d2, ob):
    'teslakit/waves.py Snell_Propagation func. parser'
    
    # get variables
    Tp = waves_dset['TPsmoo'].values[:]
    Hs = waves_dset['Hsig'].values[:]
    Dir = waves_dset['Dir'].values[:]
  
    # TODO: check dir modifications and refactor
    dir_I = Dir.copy()

    # El angulo definido entre -90 y 90
    dir_I[np.where(dir_I > 180)[0]] = dir_I[np.where(dir_I > 180)[0]] - 360

    # Obligamos que el angulo este en este sector
    dir_I[np.where(dir_I > 90)[0]] = 85
    dir_I[np.where(dir_I <= -90)[0]] = -85

    Hs_p, Dir_p, ks, kp = Snell_Propagation(Tp, Hs, dir_I, d1, d2, ob)
    
    # return copy of input
    out = waves_dset.copy()
    out['Hsig'] = Hs_p
    out['Dir'] = Dir_p
    
    return out



## SIMULATION Waves Reconstruction

In [5]:
# --------------------------------------
# RBF Reconstruction data

# historical sea
wvs_sea_dataset = db_sim.Load('sea_dataset').loc[:,['hs','tp','dir']] 
wvs_sea_subset = db_sim.Load('sea_subset').loc[:,['hs','tp','dir']] 
wvs_sea_target = db_sim.Load('sea_target').reset_index() 

# historical swells
wvs_swl_dataset = db_sim.Load('swl_dataset').loc[:,['hs','tp','dir']]  
wvs_swl_subset = db_sim.Load('swl_subset').loc[:,['hs','tp','dir']] 
wvs_swl_target = db_sim.Load('swl_target').reset_index() 

wvs_swl_target=wvs_swl_target.loc[wvs_swl_target['profile'] ==prf].reset_index().loc[:,['Hsig','TPsmoo','Dir']]
wvs_sea_target=wvs_sea_target.loc[wvs_sea_target['profile'] ==prf].reset_index().loc[:,['Hsig','TPsmoo','Dir']]


In [6]:
# --------------------------------------
# prepare dataset, subset and target data (propagated waves)

# solve Hs coefficient
wvs_sea_target['Hsig'] = wvs_sea_target['Hsig']
wvs_swl_target['Hsig'] = wvs_swl_target['Hsig'] * wvs_swl_subset['hs']

# Snell propagation 
# depth_1, depth_2, bathy_dir = 400, 21, 0
# wvs_sea_target = snell_prop_data(wvs_sea_target, depth_1, depth_2, bathy_dir)
# wvs_swl_target = snell_prop_data(wvs_swl_target, depth_1, depth_2, bathy_dir)


# remove nans (if any) from subset and target
wvs_sea_subset, wvs_sea_target = fix_target_nans(wvs_sea_subset, wvs_sea_target)
wvs_swl_subset, wvs_swl_target = fix_target_nans(wvs_swl_subset, wvs_swl_target)


# ensure dataset and subset have same variables and column order
vns_ds = ['hs', 'tp', 'dir']

wvs_sea_dataset = wvs_sea_dataset[vns_ds].values
wvs_sea_subset = wvs_sea_subset[vns_ds].values

wvs_swl_dataset = wvs_swl_dataset[vns_ds].values
wvs_swl_subset = wvs_swl_subset[vns_ds].values


# select target variables
vns_tgt = ['Hsig', 'TPsmoo', 'Dir']

wvs_sea_target = wvs_sea_target[vns_tgt].values
wvs_swl_target = wvs_swl_target[vns_tgt].values


remove nan data found at target:
     Hsig  TPsmoo  Dir
5     0.0     NaN  NaN
22    0.0     NaN  NaN
33    0.0     NaN  NaN
299   0.0     NaN  NaN


In [7]:
# --------------------------------------
# RBF Reconstruction library

# subset - scalar / directional indexes
ix_scalar_subset = [0,1]      # scalar (hs, tp)
ix_directional_subset = [2]   # directional (dir)

# target - scalar / directional indexes
ix_scalar_target = [0,1]      # scalar (Hsig, Tpsmoo, Dir)
ix_directional_target = [2]   # directional (Dir)


# RBF wrappers 
def RBF_Reconstruction_sea(dataset_input):
    
    return RBF_Reconstruction(
    wvs_sea_subset, ix_scalar_subset, ix_directional_subset,
    wvs_sea_target, ix_scalar_target, ix_directional_target,
    dataset_input)

def RBF_Reconstruction_swell(dataset_input):
    
    return RBF_Reconstruction(
    wvs_swl_subset, ix_scalar_subset, ix_directional_subset,
    wvs_swl_target, ix_scalar_target, ix_directional_target,
    dataset_input)

def RBF_Reconstruction_families(data):
   
    # sea
    vs = ['sea_Hs', 'sea_Tp', 'sea_Dir']
    data_sea = data[vs].dropna()
    data_sea.drop(data_sea[(data_sea['sea_Dir'] >= 360)].index, inplace=True)  # fix sea_Dir >> 360 bug    
    rec_sea = RBF_Reconstruction_sea(data_sea.values)
    rec_sea = pd.DataFrame(data=rec_sea, columns=vs, index=data_sea.index)

    # swell 1
    vs = ['swell_1_Hs', 'swell_1_Tp', 'swell_1_Dir']
    data_swl_1 = data[vs].dropna()
    rec_swl_1 = RBF_Reconstruction_swell(data_swl_1.values)
    rec_swl_1 = pd.DataFrame(data=rec_swl_1, columns=vs, index=data_swl_1.index)

    # swell 2
    vs = ['swell_2_Hs', 'swell_2_Tp', 'swell_2_Dir']
    data_swl_2 = data[vs].dropna()
    rec_swl_2 = RBF_Reconstruction_swell(data_swl_2.values)
    rec_swl_2 = pd.DataFrame(data=rec_swl_2, columns=vs, index=data_swl_2.index)
    
    # join nearshore reconstructed data and parse to xarray.Dataset
    rec_waves = pd.concat([rec_sea, rec_swl_1, rec_swl_2], axis=1)

    # return pandas.dataframe
    return rec_waves


In [8]:
# --------------------------------------
# Load simulation waves families

n_sims_DWTs = 10

CE = Climate_Emulator(db_S4.paths.site.EXTREMES.climate_emulator)
# CE.Load()

WVS = CE.LoadSim_All(TCs=True)
WVS

,DWT,sea_Hs,sea_Tp,sea_Dir,swell_1_Hs,swell_1_Tp,swell_1_Dir,swell_2_Hs,swell_2_Tp,swell_2_Dir,wind_speed,wind_dir,mu,tau,ss,n_sim,time
0,19.0,2.830476,7.554515,89.469269,0.485772,10.369804,303.118378,0.714205,9.351366,114.676300,10.411382,69.916573,0.000000,0.000000,0.0,0,2000-01-01 00:00:00
1,18.0,2.470053,9.519672,83.166489,2.251320,14.411101,297.476624,0.548206,16.062212,74.444191,5.499544,288.366302,0.911613,0.041667,0.0,0,2000-01-04 00:00:00
2,26.0,2.516587,8.455458,96.254684,0.921328,10.978793,339.569641,0.238640,10.769627,132.614548,8.855938,179.769669,0.700013,0.500000,0.0,0,2000-01-08 00:00:00
3,18.0,2.659001,10.644677,259.501984,0.822318,9.058253,344.315155,0.130981,17.371777,150.942169,8.749677,67.079559,0.927076,0.875000,0.0,0,2000-01-11 00:00:00
4,35.0,2.108193,8.055782,353.234863,0.327759,10.712851,212.896820,0.038747,13.807611,85.415718,7.890670,81.755096,0.921430,0.041667,0.0,0,2000-01-17 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235040,21.0,1.491215,9.590439,34.318424,0.717192,10.582491,36.022396,2.336690,8.537787,151.475525,5.568329,60.044518,0.840002,0.097222,0.0,9,2999-12-21 00:00:00
1235041,18.0,2.276567,8.808626,80.724442,0.401999,9.572033,300.671295,0.050406,11.165343,128.718964,4.865163,107.378662,0.863095,0.020833,0.0,9,2999-12-22 00:00:00
1235042,32.0,3.314268,9.863656,83.455490,0.367855,10.029246,354.611633,0.181040,9.582941,72.150703,7.675133,244.395218,0.780410,0.041667,0.0,9,2999-12-30 00:00:00
1235043,18.0,1.441129,9.059766,77.198402,1.802036,10.657471,16.481543,0.589877,15.243070,81.542709,1.989503,86.757362,0.000000,0.000000,0.0,9,2999-12-31 00:00:00


In [9]:
# Reconstruct waves by families
WVS_rec = RBF_Reconstruction_families(WVS)

# set indexes values
WVS_rec['n_sim'] = WVS.n_sim
WVS_rec['time'] = WVS.time
WVS_rec['wind_speed'] = WVS.wind_speed
WVS_rec['wind_dir'] = WVS.wind_dir
WVS_rec['mu'] = WVS.mu
WVS_rec['tau'] = WVS.tau
WVS_rec['ss'] = WVS.ss

# store reconstruction
# db_sim_S4.Save('reconstruction_prof'+ str(prf), WVS_rec)

ix_scalar: 0,  optimization: 28.82 | interpolation: 63.68
ix_scalar: 1,  optimization: 34.18 | interpolation: 65.99
ix_directional: 2,  optimization: 60.43 | interpolation: 127.09
ix_scalar: 0,  optimization: 9.89 | interpolation: 61.40
ix_scalar: 1,  optimization: 28.34 | interpolation: 64.93
ix_directional: 2,  optimization: 33.61 | interpolation: 130.14
ix_scalar: 0,  optimization: 10.56 | interpolation: 60.98
ix_scalar: 1,  optimization: 28.60 | interpolation: 63.92
ix_directional: 2,  optimization: 33.69 | interpolation: 125.78


In [10]:
key=os.path.join(db_sim_S4.p_base,'Reconstruction_profile' + str(prf))
WVS_rec.to_pickle(key)
print ('Profile '+ str(prf)+' --> Saved in: ' + key)

Profile 11 --> Saved in: /media/administrador/HD/Dropbox/Guam/teslakit/data/sites/GUAM_CC_S4/HYSWAN/sim/Reconstruction_profile11


In [11]:
#We change the funcion from teslakit.database to include profile number

from teslakit.database import SplitStorage

def Save_SIM_NEARSHORE(self, xds, n_sim, prf):
    ps = self.paths.site.SIMULATION.nearshore

    nm = '{0:08d}'.format(n_sim)  # sim code
    ps_sim = op.join(ps, nm + '_prf_' + str(prf))

    s =  SplitStorage(ps_sim)
    s.Save(xds, safe_time=True)

In [12]:
def waves_hourly(CE, WVS_SIM_rec, n_sim_ce=0, db=[], prf=0):
    '''
    Generate nearshore hourly simulation output. 
    
    - aggregate nearshore waves storms
    - generate hourly hydrographs
    - calculates AWL
    
    Stores simulation nearshore waves at teslakit site database
    '''
    
    # iterate simulations
    for n in range(n_sims_DWTs):
        
        # select simulation
        WVS_SIM_rec_n = WVS_SIM_rec.loc[WVS_SIM_rec['n_sim'] == n]

        # aggregate nearshore variables (Tp from max. energy waves family)
        wvs_agr = Aggregate_WavesFamilies(WVS_SIM_rec_n, a_tp='max_energy')                
        
        # Load TCs storm data from climate emulator
        _, TCS_sim, _ = CE.LoadSim(n_sim = n)    
        TCS_sim = TCS_sim.isel(n_sim = n_sim_ce).copy() 
        TCS_sim = TCS_sim.sel(time=wvs_agr.time)
        
        # calculate intradaily hourly hydrographs for simulated storms
        wvs_h = Intradaily_Hydrograph(wvs_agr, TCS_sim)
        
        # calculate AWL
        wvs_h['AWL'] = AWL(wvs_h.Hs, wvs_h.Tp)
        
        # store hourly simulation offshore data
#         db.Save_SIM_NEARSHORE(wvs_h, n)
        Save_SIM_NEARSHORE(db, wvs_h, n, prf)
                
        print('simulation {0} processed.'.format(n))

# generate intradaily hydrographs


In [13]:
waves_hourly(CE, WVS_rec, db=db_S4, prf=prf)  

simulation 0 processed.
simulation 1 processed.
simulation 2 processed.
simulation 3 processed.
simulation 4 processed.
simulation 5 processed.
simulation 6 processed.
simulation 7 processed.
simulation 8 processed.
simulation 9 processed.
